In [1]:
import pandas as pd
import sqlite3
import os
from IPython.display import HTML, display
import glob
from datetime import datetime
import us
import csv

### File Prep: ###
* Read in file, skipping over top 7 rows
* Convert 'order postal' to category dtype (first experiment with this dtype, look for weirdness...)
* Load function to clean column names (used as db is being created)

In [16]:
client = 'test_client'

encode = 'latin_1'
skip_rows = 7
df = pd.read_csv('test-sales.csv', encoding = encode, skiprows=skip_rows, dtype={'order postal': 'str'})

# Would like to play with pandas dtype 'category' but breaks the zipcode string operations so for future development:
# df = pd.read_csv('/Users/karen/Documents/Business/Agile-Merchant/clients/spontuneous/data/2017-10-17/2016Feb1-2017Oct17CustomTransaction.csv', encoding = encode, skiprows=skip_rows, dtype={'order postal': 'category'})

display(df.head(10))

,date/time,settlement id,type,order id,sku,quantity,order city,order state,order postal,product sales,other,total
0,"Feb 1, 2016 12:28:08 AM PST",6048826701,Order,105-0254632-8301849,sku01,1.0,MARION,OH,43302-5231,35.00,0,26.12
1,"Feb 1, 2016 2:09:21 PM PST",6048826701,Refund,103-9249887-1536265,sku02,1.0,TENAFLY,NJ,07670-3014,-32.99,0,-29.03
2,"Feb 1, 2016 3:19:52 PM PST",6048826701,Order,104-3152419-5177066,sku02,1.0,MILILANI,HI,96789-2046,32.99,0,24.02
3,"Feb 1, 2016 3:35:39 PM PST",6048826701,Order,114-6293518-1467419,sku03,1.0,BLOOMFIELD,MO,63825-9505,16.99,0,11.77
4,"Feb 1, 2016 5:49:49 PM PST",6048826701,Order,113-4059732-1893026,sku01,1.0,CLIFTON,NJ,07011-3215,35.00,0,26.12
5,"Feb 3, 2016 1:52:11 AM PST",6048826701,Order,112-4824706-5214658,sku02,1.0,REDMOND,WA,98052-6122,32.99,0,24.02
6,"Feb 3, 2016 1:54:22 AM PST",6048826701,Order,110-6682296-7137053,sku02,1.0,NASHVILLE,TN,37212-4810,32.99,0,24.02
7,"Feb 3, 2016 2:17:18 AM PST",6048826701,Order,105-8169048-3253065,sku01,1.0,LAKE ZURICH,IL,60047-2839,35.00,0,26.12
8,"Feb 3, 2016 1:40:23 PM PST",6048826701,Order,002-5599967-6321059,sku04,1.0,ALEXANDRIA,VA,22314-4755,49.99,0,33.73
9,"Feb 4, 2016 8:28:33 AM PST",6048826701,Order,102-2342223-6180238,sku01,1.0,KELLER,TEXAS,76248-4148,35.00,0,26.12


In [5]:
def clean_col_names(df):
    characters = (' ', '/', '-')
    for char in characters:
        # make column names more sql friendly
        df.columns = df.columns.str.replace(char,'_')
        display(df.columns)
    return df

### Create toy SQLite db

In [6]:
db_file = '{}-all-transactions.sqlite'.format(client) # this would populate the client or variable name from the client variable cell
conn = sqlite3.connect( db_file )
c = conn.cursor()

Import data into SQLite db:

In [7]:
def create_table(df,table_name):
    df.to_sql( table_name, conn, index = False, if_exists = 'replace' )

Create tables based on available dataframes and transform column names

In [8]:
# Set up a dictionary of Dataframe names and assosiated SQLite table names.
table_map = [
    (df,'transactions')
    # Add additional dfs and table names here as needed
]

# Create a table for each dataframe in the table_map.
for table in table_map:
    df = table[0]
    table_name = table[1]
    # make column names more sql friendly, this should be moved out of this function, not associated with the SQLite DB only.
    clean_col_names(df)
    create_table(df, table_name)

Index(['date/time', 'settlement_id', 'type', 'order_id', 'sku', 'quantity',
       'order_city', 'order_state', 'order_postal', 'product_sales', 'other',
       'total'],
      dtype='object')

Index(['date_time', 'settlement_id', 'type', 'order_id', 'sku', 'quantity',
       'order_city', 'order_state', 'order_postal', 'product_sales', 'other',
       'total'],
      dtype='object')

Index(['date_time', 'settlement_id', 'type', 'order_id', 'sku', 'quantity',
       'order_city', 'order_state', 'order_postal', 'product_sales', 'other',
       'total'],
      dtype='object')

## Start cleaning data
User-entered data: Foibles ahead.

### Normalize state names:
* Create a list of all unique values in "order_state" column
* Iterate through the list, determine which states are in the US states library
* If state exists, update table with normalized name, otherwise pass (this maintains non-US place values)

*Note:  This section creates a second cursor, ```update_c```, just for use in the if statement to update the table. The ```c``` cursor is the lookup cursor, so can't update itself.*

In [9]:
# Create a database connection and cursor object
db_file = '{}-all-transactions.sqlite'.format(client) # this would populate the client or variable name from the client variable cell
conn = sqlite3.connect( db_file )
c = conn.cursor()
sales_state = []

# Replace any "." in the state names. This confuses the state lookup library
c.execute('update transactions set order_state = replace(order_state,".","")')

# Create the list of states that will be used to look up the normalized state names.
all_state_values = c.execute('select distinct order_state from transactions where order_state != ""')

# Iterate through the list of states, look up whether it's a US state, and update with normalized version of name
for value in all_state_values:
    raw_state = (value[0])
    
#     print(raw_state)  # This line is for visually verifying the state being changed
    new_state = str(us.states.lookup(raw_state))
    if new_state != 'None': 
#         print(new_state)  # This line is also for visually verifying the state being changed ^^^
        sales_state.append(new_state)
        #display(sales_state) # More visual testing...
        update_c = conn.cursor() # Create update-specific cursor, select cursor can't update its own data
        update_c.execute('update transactions set order_state = ? where order_state = ?',(new_state,raw_state))
        
    else:
        continue

# Test to verify states have been updated, non-US states left in place
new_state_values = c.execute('select distinct order_state from transactions')
display(new_state_values.fetchall())        

# Commit the changes and close the database connection
conn.commit()
conn.close()


[('Ohio',),
 ('New Jersey',),
 ('Hawaii',),
 ('Missouri',),
 ('Washington',),
 ('Tennessee',),
 ('Illinois',),
 ('Virginia',),
 ('Texas',),
 ('Florida',),
 ('New York',),
 ('Connecticut',),
 ('Pennsylvania',),
 ('California',),
 ('Colorado',),
 ('Oregon',),
 ('Wisconsin',),
 ('Manitoba',),
 ('cornwall',),
 ('Worcs',),
 ('New Brunswick',),
 ('BC',),
 ('Ontario',),
 ('NL',),
 ('Roma',),
 (None,),
 ('Nagaland',),
 ('South Carolina',),
 ('Arkansas',),
 ('Iowa',),
 ('Arizona',),
 ('Alabama',),
 ('Maryland',),
 ('Montana',),
 ('Utah',),
 ('W Sussex',),
 ('Glos',),
 ('North Carolina',),
 ('Michigan',),
 ('Minnesota',),
 ('Kentucky',),
 ('Indiana',),
 ('Georgia',),
 ('New Hampshire',),
 ('Kansas',),
 ('Oklahoma',),
 ('Nova Scotia',),
 ('Massachusetts',),
 ('North Dakota',),
 ('Louisiana',),
 ('Maine',),
 ('West Virginia',),
 ('Nevada',),
 ('Idaho',),
 ('Alaska',),
 ('Mississippi',),
 ('New Mexico',),
 ('Nebraska',),
 ('Hampshire',),
 ('AP',),
 ('Delaware',),
 ('Wyoming',),
 ('Guam',),
 ('South

### Clean Zipcodes ###
Create less granular groups and more interesting visualization by shortening to 5 digit zips.

00000-0000 -> 00000

Future: 
* This is kind of blunt because it just slices anything longer than 5 characters. What about non-US postal codes?
* Perhaps only look for '-' in index 5 and slice then? Again, non-US consideration.
* Would like to play with dtype category, but breaks because of NaNs. To work on in the future.

[Resource on zips and dtype for leading zeros](http://data-tutorials.com/zip-codes-in-pandas.html)

[Current pandas doc on dtype 'category', article above is out of date here](https://pandas.pydata.org/pandas-docs/stable/categorical.html)

In [11]:
# A quick helper to test the number of postal codes that have '-'. Should run both before and after the string operation
def get_dashes(df):
    long_postals = df[df['order_postal'].str.contains('-', na = False)]
    dash_count = len(long_postals['order_postal'])
    return(dash_count, long_postals)

# Slices the zipcode based on length:
def clean_dashes(df):
    long_zip_codes = df['order_postal'].str.len() > 5
    df['order_postal'] = df['order_postal'].str.slice(0, 5)# Don't love this, would rather slice after the '-' character? What about non-US postal codes?
    
# Total row count to get a basesline for rows in the set:
row_count = len(df['order_postal'])
display('total dataframe rows: ',row_count)

# Rows found to have dashes:
raw_row_count, raw_long_postals = get_dashes(df)
display("Raw data: ", 'count: ',raw_row_count, 'zips with dash: ',raw_long_postals['order_postal'].unique())
# Now clean up the zips that have dashes:
if raw_row_count > 0: 
    clean_dashes(df)
else: 
    display('no long zips')

# Re-check for dashes to verify changes:   
cleaned_long_row_count, cleaned_long_postals = get_dashes(df)
display("Cleaned data: ", 'count: ',cleaned_long_row_count, 'zips with dash: ',cleaned_long_postals['order_postal'].unique())

    
# Some alt code to test and incorporate:
# long_zip_codes = df['order postal'].str.len() > 5
# df['order postal'][long_zip_codes].unique()

'total dataframe rows: '

49760

'Raw data: '

'count: '

36874

...

'Cleaned data: '

'count: '

2

'zips with dash: '

array(['106-0'], dtype=object)

### Summarize sales data by state

To Do:
* Convert sales to int64 instead of float
* Add sales by zip code
* Found repeats and problems with city data. Clean it up!

In [12]:
# Create a database connection and cursor object
conn = sqlite3.connect( db_file )
c = conn.cursor()

# Create a unique list of US states that have sales:
state_set = list(set(sales_state))

# Iterate through the state list and sum sales by state:
sales_by_state = []
# sales_by_zip = []
for state in state_set:
    c.execute('select ?, sum(quantity) from transactions where type is "Order" and order_state is ?;',(state,state))
    sales_by_state.append(c.fetchone())
# sales_by_state = dict(sales_by_state)

display(sales_by_state)

conn.commit()
conn.close()

[('Ohio', 1873.0),
 ('Georgia', 1236.0),
 ('Kansas', 508.0),
 ('Guam', 4.0),
 ('Vermont', 124.0),
 ('Northern Mariana Islands', 2.0),
 ('Tennessee', 811.0),
 ('Nebraska', 386.0),
 ('South Carolina', 526.0),
 ('Arizona', 1003.0),
 ('Puerto Rico', 24.0),
 ('Florida', 2161.0),
 ('Oklahoma', 437.0),
 ('Wisconsin', 978.0),
 ('New Mexico', 160.0),
 ('Iowa', 536.0),
 ('Connecticut', 822.0),
 ('North Dakota', 153.0),
 ('New Hampshire', 359.0),
 ('Arkansas', 292.0),
 ('Colorado', 1092.0),
 ('North Carolina', 1339.0),
 ('Kentucky', 558.0),
 ('Washington', 1389.0),
 ('West Virginia', 217.0),
 ('Idaho', 366.0),
 ('Louisiana', 486.0),
 ('Utah', 1136.0),
 ('Mississippi', 221.0),
 ('South Dakota', 129.0),
 ('Oregon', 635.0),
 ('Maine', 249.0),
 ('Minnesota', 914.0),
 ('Massachusetts', 1627.0),
 ('Alaska', 113.0),
 ('Wyoming', 124.0),
 ('District of Columbia', 92.0),
 ('Michigan', 1452.0),
 ('Rhode Island', 160.0),
 ('Virgin Islands', 1.0),
 ('Pennsylvania', 2230.0),
 ('Virginia', 1661.0),
 ('Maryland

Create csv of total sales by city:
* SQL query selects city and sum quantities only for orders (not returns etc) and groups the output by city


In [13]:

# Create a database connection and cursor object
conn = sqlite3.connect( db_file )
c = conn.cursor()

# Sum sales by city:
c.execute('select order_state, order_city, sum(quantity) from transactions where type is "Order" group by order_state, order_city;')

# Export to csv:
with open("sales_totals.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(['order_state', 'order_city', 'quantity'])
    writer.writerows(c.fetchall())


### Clean City###
Dataset contains multiple versions of the same city because of case differences so convert to all caps.


### Problems here: This isn't working, unique count value before and after cleaning is the same. Why? ###

In [ ]:
new_df = pd.DataFrame(df)
new_df['order city'] = new_df['order city'].str.upper()
display(df)
display(new_df)
display(len(new_df['order city'].unique()),len(df['order city'].unique()))
# df['order city'].fillna(False)

# len(all_names['order city'].unique())
# all_caps_df = df['order city'].str.upper()
# len(all_caps_df['order city'].unique())

# print(df.columns.values)

Helper code to force-close the database when it gets stuck locked:

In [ ]:
conn.commit()
conn.close()

For later mapping and pretty display, here's a [link to a dictionary of abbreviations and state names](http://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/).